# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 13 2022 2:06PM,241780,20,ALU0007306,Alumier Labs Inc.,Released
1,Jul 13 2022 2:05PM,241789,19,ADV80004151,"AdvaGen Pharma, Ltd",Released
2,Jul 13 2022 2:05PM,241789,19,ADV80004152,"AdvaGen Pharma, Ltd",Released
3,Jul 13 2022 2:05PM,241789,19,ADV80004153,"AdvaGen Pharma, Ltd",Released
4,Jul 13 2022 2:04PM,241788,10,Enova-10293,Emerginnova,Released
5,Jul 13 2022 2:04PM,241788,10,Enova-10294,Emerginnova,Released
6,Jul 13 2022 2:04PM,241788,10,Enova-10295,Emerginnova,Released
7,Jul 13 2022 2:04PM,241788,10,Enova-10296,Emerginnova,Released
8,Jul 13 2022 1:55PM,241787,16,TASA-277112,TASA USA Inc.,Released
9,Jul 13 2022 1:27PM,241786,21,8SAMPLE-403040419,"NBTY Global, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,241785,Executing,1
35,241786,Released,1
36,241787,Released,1
37,241788,Released,4
38,241789,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241785,NaN,1.0,NaN
241786,NaN,NaN,1.0
241787,NaN,NaN,1.0
241788,NaN,NaN,4.0
241789,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241705,0.0,0.0,1.0
241706,0.0,1.0,0.0
241710,14.0,11.0,1.0
241712,0.0,0.0,6.0
241723,0.0,11.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241705,0,0,1
241706,0,1,0
241710,14,11,1
241712,0,0,6
241723,0,11,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241705,0,0,1
1,241706,0,1,0
2,241710,14,11,1
3,241712,0,0,6
4,241723,0,11,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241705,,,1
1,241706,,1,
2,241710,14,11,1
3,241712,,,6
4,241723,,11,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 13 2022 2:06PM,241780,20,Alumier Labs Inc.
1,Jul 13 2022 2:05PM,241789,19,"AdvaGen Pharma, Ltd"
4,Jul 13 2022 2:04PM,241788,10,Emerginnova
8,Jul 13 2022 1:55PM,241787,16,TASA USA Inc.
9,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc."
10,Jul 13 2022 1:26PM,241785,20,Else Nutrition
11,Jul 13 2022 1:23PM,241784,21,"NBTY Global, Inc."
12,Jul 13 2022 1:13PM,241782,20,Alumier Labs Inc.
26,Jul 13 2022 1:05PM,241779,19,"GCH Granules USA, Inc."
27,Jul 13 2022 1:03PM,241775,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 13 2022 2:06PM,241780,20,Alumier Labs Inc.,,,1
1,Jul 13 2022 2:05PM,241789,19,"AdvaGen Pharma, Ltd",,,3
2,Jul 13 2022 2:04PM,241788,10,Emerginnova,,,4
3,Jul 13 2022 1:55PM,241787,16,TASA USA Inc.,,,1
4,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc.",,,1
5,Jul 13 2022 1:26PM,241785,20,Else Nutrition,,1,
6,Jul 13 2022 1:23PM,241784,21,"NBTY Global, Inc.",,,1
7,Jul 13 2022 1:13PM,241782,20,Alumier Labs Inc.,,,14
8,Jul 13 2022 1:05PM,241779,19,"GCH Granules USA, Inc.",,1,
9,Jul 13 2022 1:03PM,241775,10,"Methapharm, Inc.",,,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 2:06PM,241780,20,Alumier Labs Inc.,1,,
1,Jul 13 2022 2:05PM,241789,19,"AdvaGen Pharma, Ltd",3,,
2,Jul 13 2022 2:04PM,241788,10,Emerginnova,4,,
3,Jul 13 2022 1:55PM,241787,16,TASA USA Inc.,1,,
4,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc.",1,,
5,Jul 13 2022 1:26PM,241785,20,Else Nutrition,,1,
6,Jul 13 2022 1:23PM,241784,21,"NBTY Global, Inc.",1,,
7,Jul 13 2022 1:13PM,241782,20,Alumier Labs Inc.,14,,
8,Jul 13 2022 1:05PM,241779,19,"GCH Granules USA, Inc.",,1,
9,Jul 13 2022 1:03PM,241775,10,"Methapharm, Inc.",16,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 2:06PM,241780,20,Alumier Labs Inc.,1,,
1,Jul 13 2022 2:05PM,241789,19,"AdvaGen Pharma, Ltd",3,,
2,Jul 13 2022 2:04PM,241788,10,Emerginnova,4,,
3,Jul 13 2022 1:55PM,241787,16,TASA USA Inc.,1,,
4,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 2:06PM,241780,20,Alumier Labs Inc.,1.0,NaN,NaN
1,Jul 13 2022 2:05PM,241789,19,"AdvaGen Pharma, Ltd",3.0,NaN,NaN
2,Jul 13 2022 2:04PM,241788,10,Emerginnova,4.0,NaN,NaN
3,Jul 13 2022 1:55PM,241787,16,TASA USA Inc.,1.0,NaN,NaN
4,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 2:06PM,241780,20,Alumier Labs Inc.,1.0,0.0,0.0
1,Jul 13 2022 2:05PM,241789,19,"AdvaGen Pharma, Ltd",3.0,0.0,0.0
2,Jul 13 2022 2:04PM,241788,10,Emerginnova,4.0,0.0,0.0
3,Jul 13 2022 1:55PM,241787,16,TASA USA Inc.,1.0,0.0,0.0
4,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2659347,60.0,18.0,1.0
12,241771,1.0,0.0,0.0
15,483422,7.0,11.0,14.0
16,1208717,2.0,23.0,0.0
19,725308,4.0,1.0,0.0
20,1208845,27.0,4.0,0.0
21,966981,3.0,1.0,0.0
22,483493,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2659347,60.0,18.0,1.0
1,12,241771,1.0,0.0,0.0
2,15,483422,7.0,11.0,14.0
3,16,1208717,2.0,23.0,0.0
4,19,725308,4.0,1.0,0.0
5,20,1208845,27.0,4.0,0.0
6,21,966981,3.0,1.0,0.0
7,22,483493,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,60.0,18.0,1.0
1,12,1.0,0.0,0.0
2,15,7.0,11.0,14.0
3,16,2.0,23.0,0.0
4,19,4.0,1.0,0.0
5,20,27.0,4.0,0.0
6,21,3.0,1.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,60.0
1,12,Released,1.0
2,15,Released,7.0
3,16,Released,2.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,1.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0
Executing,18.0,0.0,11.0,23.0,1.0,4.0,1.0,0.0
Released,60.0,1.0,7.0,2.0,4.0,27.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0
1,Executing,18.0,0.0,11.0,23.0,1.0,4.0,1.0,0.0
2,Released,60.0,1.0,7.0,2.0,4.0,27.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0
1,Executing,18.0,0.0,11.0,23.0,1.0,4.0,1.0,0.0
2,Released,60.0,1.0,7.0,2.0,4.0,27.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()